In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pygplates
import pygmt

from gprm import ReconstructionModel
from gprm.datasets import Rocks, Reconstructions, Paleogeography, Geology
from gprm.utils.geometry import nearest_feature

import sys
#sys.path.append('/Users/simon/OneDrive/Andes_works//python/')
sys.path.append('../../andes_paper/python/')
import joyful_geochemistry as joy
import joyful_mapping as joymap
from time_series import lowess

import collections
import matplotlib.ticker as mticker

%matplotlib inline
%load_ext autoreload
%autoreload 2



In [ ]:
topo = pygmt.datasets.load_earth_relief(resolution='06m')

TectonicMap = Geology.fetch_GlobalTectonicMap()

In [ ]:
TectonicMap.plot(column='lastorogen')
#TectonicMap.columns

In [ ]:
fig=pygmt.Figure()
fig.basemap(frame='fg', projection='N15c', region='d')
#fig.coast(land='gray', projection='N15c', region='d')
pygmt.makecpt(cmap='gray', series=[-1000,5000,500], background='o', reverse=True)
fig.grdimage(grid=topo, cmap=True)
#fig.plot(data=TectonicMap.dropna(subset=['lastorogen']), fill='blue', transparency=50)

orogen_name = 'Cordilleran'
fig.plot(data=TectonicMap.dropna(subset='lastorogen').query('lastorogen.str.match(@orogen_name)'), 
         fill='blue', transparency=80)
fig.show()




In [ ]:
orogen_names = [
    ('Cordilleran', 'Blue'), 
    ('Terra Australis', 'Green'), 
    ('Solonker', 'Orange'), 
    ('Indosinian', 'Red'), 
    ('Variscan-Hercynian', 'DarkGreen'),
    #'Scotian',
    #'Uralian',
    ('Mongol', 'Purple'),
    ('Kazakh', 'Cyan'),
    ('Caledonia', 'DeepPink'),
    ('Alpine-Himalayan', 'Gold')
]

fig=pygmt.Figure()
fig.basemap(frame='fg', projection='N22c', region='d')
#fig.coast(land='gray', projection='N15c', region='d')
pygmt.makecpt(cmap='gray', series=[-1000,5000,500], background='o', reverse=True)
fig.grdimage(grid=topo, cmap=True)
#fig.plot(data=TectonicMap.dropna(subset=['lastorogen']), fill='blue', transparency=50)

for orogen_name, color in orogen_names:
    fig.plot(data=TectonicMap.dropna(subset='lastorogen').query('lastorogen.str.match(@orogen_name)'), 
             fill=color, #pen='1p,white', 
             transparency=70, label=orogen_name)
fig.legend(position='JBL+jBL+o-0.1c', box='+gwhite+p1p')
fig.show(width=800)


In [ ]:
df = pd.read_csv('../datafiles/geochem_global_20230124_subduction_M2021.csv')
df = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df['geometry']), crs='EPSG:4326')

sz_distance_limit = 750.

df = df.query('distance_to_sz<@sz_distance_limit & age<=540').reset_index(drop=True)

fig=pygmt.Figure()
fig.basemap(frame='fg', projection='N22c', region='d')
#fig.coast(land='gray', projection='N15c', region='d')
pygmt.makecpt(cmap='gray', series=[-1000,5000,500], background='o', reverse=True)
fig.grdimage(grid=topo, cmap=True)
#fig.plot(data=TectonicMap.dropna(subset=['lastorogen']), fill='blue', transparency=50)

fig.plot(data=df, style='c0.1c', fill='black', pen='0.1p,white')

for orogen_name, color in orogen_names:
    fig.plot(data=TectonicMap.dropna(subset='lastorogen').query('lastorogen.str.match(@orogen_name)'), 
             fill=color, #pen='1p,white', 
             transparency=70, label=orogen_name)
    
    
fig.legend(position='JBL+jBL+o-0.1c', box='+gwhite+p1p')
fig.savefig('../images/global_orogens.png', dpi=600)
fig.show(width=800)


In [ ]:

fig=pygmt.Figure()
fig.basemap(frame='fg', projection='N15c', region='d')
#fig.coast(land='gray', projection='N15c', region='d')
pygmt.makecpt(cmap='gray', series=[-1000,5000,500], background='o', reverse=True)
fig.grdimage(grid=topo, cmap=True)
#fig.plot(data=TectonicMap.dropna(subset=['lastorogen']), fill='blue', transparency=50)

orogen_name = 'Cordilleran'
#continent_name = 'South America'
#continent_name = 'North America'
continent_name = 'Asia'
data = TectonicMap.dropna(subset=['lastorogen','continent']).query('lastorogen.str.match(@orogen_name)').query('continent.str.match(@continent_name)')

fig.plot(data=data, 
         fill='blue', transparency=80)
fig.show()


In [ ]:
TectonicMap.continent.unique()

In [ ]:
Paleomap = Reconstructions.fetch_Scotese()

orogens_pmap = Paleomap.assign_plate_ids(TectonicMap)

orogens_pmap.plot()

In [ ]:
#'|'.join(list(zip(*orogen_names))[0])
orogens_pmap = TectonicMap.dropna(subset=['lastorogen'])
orogens_pmap = orogens_pmap[orogens_pmap.lastorogen.str.contains('|'.join(list(zip(*orogen_names))[0]))]

orogens_pmap = Paleomap.assign_plate_ids(orogens_pmap)

orogens_pmap.plot(column='PLATEID1')

In [ ]:
pmap_gdf = gpd.read_file(Paleomap.static_polygon_files[0])
pmap_gdf.plot()
